# Mamba-ETTm1 Forecasting on Google Colab

This notebook sets up and runs the project on Colab with GPU support.

**Setup:** Runtime → Change runtime type → GPU


In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU found")


In [ ]:
# Clone repository
!git clone https://github.com/steinpleiter/Mamba-ETTm1-Forecasting.git
%cd Mamba-ETTm1-Forecasting


In [ ]:
!pip install -q pandas numpy scikit-learn matplotlib seaborn pyyaml tqdm
!pip install -q causal-conv1d>=1.1.0
!pip install -q mamba-ssm>=1.2.0
print("All dependencies installed!")


In [ ]:
# Download ETTm1 dataset
import os, urllib.request
os.makedirs('data/raw', exist_ok=True)
url = 'https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTm1.csv'
urllib.request.urlretrieve(url, 'data/raw/ETTm1.csv')
print("Dataset downloaded!")


In [ ]:
# Preprocess data
!python scripts/preprocess_data.py --config configs/base_config.yaml


In [ ]:
# Train DLinear (channel-independent mode)
!python scripts/train_dlinear.py --device cuda --individual --epochs 50 --batch_size 64


In [ ]:
# Evaluate trained model
!python scripts/evaluate_trained_model.py --device cuda


In [ ]:
!zip -r results.zip results/
from google.colab import files
files.download('results.zip')
print("Download results.zip")
